In [1]:
import numpy as np
import scipy as sp
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import geopandas as gpd

/Users/lilyli/opt/anaconda3/lib/python3.9/site-packages/geopandas/_compat.py:112: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [2]:
sf = gpd.read_file("../data/ABS/SA2 shapefile/SA2_2021_AUST_GDA2020.shp")
sf = sf.drop(2472)
sf

,SA2_CODE21,SA2_NAME21,CHG_FLAG21,CHG_LBL21,SA3_CODE21,SA3_NAME21,SA4_CODE21,SA4_NAME21,GCC_CODE21,GCC_NAME21,STE_CODE21,STE_NAME21,AUS_CODE21,AUS_NAME21,AREASQKM21,LOCI_URI21,geometry
0,101021007,Braidwood,0,No change,10102,Queanbeyan,101,Capital Region,1RNSW,Rest of NSW,1,New South Wales,AUS,Australia,3418.3525,http://linked.data.gov.au/dataset/asgsed3/SA2/...,"POLYGON ((149.58424 -35.44426, 149.58444 -35.4..."
1,101021008,Karabar,0,No change,10102,Queanbeyan,101,Capital Region,1RNSW,Rest of NSW,1,New South Wales,AUS,Australia,6.9825,http://linked.data.gov.au/dataset/asgsed3/SA2/...,"POLYGON ((149.21899 -35.36738, 149.21800 -35.3..."
2,101021009,Queanbeyan,0,No change,10102,Queanbeyan,101,Capital Region,1RNSW,Rest of NSW,1,New South Wales,AUS,Australia,4.7620,http://linked.data.gov.au/dataset/asgsed3/SA2/...,"POLYGON ((149.21326 -35.34325, 149.21619 -35.3..."
3,101021010,Queanbeyan - East,0,No change,10102,Queanbeyan,101,Capital Region,1RNSW,Rest of NSW,1,New South Wales,AUS,Australia,13.0032,http://linked.data.gov.au/dataset/asgsed3/SA2/...,"POLYGON ((149.24034 -35.34781, 149.24024 -35.3..."
4,101021012,Queanbeyan West - Jerrabomberra,0,No change,10102,Queanbeyan,101,Capital Region,1RNSW,Rest of NSW,1,New South Wales,AUS,Australia,13.6748,http://linked.data.gov.au/dataset/asgsed3/SA2/...,"POLYGON ((149.19572 -35.36126, 149.19970 -35.3..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2467,901021002,Cocos (Keeling) Islands,0,No change,90102,Cocos (Keeling) Islands,901,Other Territories,9OTER,Other Territories,9,Other Territories,AUS,Australia,13.7258,http://linked.data.gov.au/dataset/asgsed3/SA2/...,"MULTIPOLYGON (((96.91512 -12.14044, 96.91513 -..."
2468,901031003,Jervis Bay,0,No change,90103,Jervis Bay,901,Other Territories,9OTER,Other Territories,9,Other Territories,AUS,Australia,67.2296,http://linked.data.gov.au/dataset/asgsed3/SA2/...,"MULTIPOLYGON (((150.69567 -35.18295, 150.69556..."
2469,901041004,Norfolk Island,0,No change,90104,Norfolk Island,901,Other Territories,9OTER,Other Territories,9,Other Territories,AUS,Australia,38.6510,http://linked.data.gov.au/dataset/asgsed3/SA2/...,"MULTIPOLYGON (((167.96325 -29.07212, 167.96326..."
2470,997979799,Migratory - Offshore - Shipping (OT),0,No change,99797,Migratory - Offshore - Shipping (OT),997,Migratory - Offshore - Shipping (OT),99799,Migratory - Offshore - Shipping (OT),9,Other Territories,AUS,Australia,NaN,http://linked.data.gov.au/dataset/asgsed3/SA2/...,None


In [3]:
gdf = gpd.GeoDataFrame(sf)
gdf['SA2_CODE21'] = gdf['SA2_CODE21'].astype('float')
gdf.head()

,SA2_CODE21,SA2_NAME21,CHG_FLAG21,CHG_LBL21,SA3_CODE21,SA3_NAME21,SA4_CODE21,SA4_NAME21,GCC_CODE21,GCC_NAME21,STE_CODE21,STE_NAME21,AUS_CODE21,AUS_NAME21,AREASQKM21,LOCI_URI21,geometry
0,101021007.0,Braidwood,0,No change,10102,Queanbeyan,101,Capital Region,1RNSW,Rest of NSW,1,New South Wales,AUS,Australia,3418.3525,http://linked.data.gov.au/dataset/asgsed3/SA2/...,"POLYGON ((149.58424 -35.44426, 149.58444 -35.4..."
1,101021008.0,Karabar,0,No change,10102,Queanbeyan,101,Capital Region,1RNSW,Rest of NSW,1,New South Wales,AUS,Australia,6.9825,http://linked.data.gov.au/dataset/asgsed3/SA2/...,"POLYGON ((149.21899 -35.36738, 149.21800 -35.3..."
2,101021009.0,Queanbeyan,0,No change,10102,Queanbeyan,101,Capital Region,1RNSW,Rest of NSW,1,New South Wales,AUS,Australia,4.7620,http://linked.data.gov.au/dataset/asgsed3/SA2/...,"POLYGON ((149.21326 -35.34325, 149.21619 -35.3..."
3,101021010.0,Queanbeyan - East,0,No change,10102,Queanbeyan,101,Capital Region,1RNSW,Rest of NSW,1,New South Wales,AUS,Australia,13.0032,http://linked.data.gov.au/dataset/asgsed3/SA2/...,"POLYGON ((149.24034 -35.34781, 149.24024 -35.3..."
4,101021012.0,Queanbeyan West - Jerrabomberra,0,No change,10102,Queanbeyan,101,Capital Region,1RNSW,Rest of NSW,1,New South Wales,AUS,Australia,13.6748,http://linked.data.gov.au/dataset/asgsed3/SA2/...,"POLYGON ((149.19572 -35.36126, 149.19970 -35.3..."


In [4]:
# create a JSON 
geoJSON = gdf[['SA2_CODE21', 'geometry']].drop_duplicates('SA2_CODE21').to_json()

In [5]:
import folium


In [6]:
df = pd.read_csv('../data/ABS/clean_population.csv')

df_population = df[['SA2_code', '2021']]
df_population.columns = ['SA2_code', 'population']
df_population = df_population[df_population['population'] < 30000]
df_population

,SA2_code,population
0,101021007.0,4330.0
1,101021008.0,8546.0
2,101021009.0,11370.0
3,101021010.0,5093.0
4,101021012.0,12743.0
...,...,...
2449,801111141.0,67.0
2450,901011001.0,1716.0
2451,901021002.0,602.0
2452,901031003.0,310.0


In [7]:
_map = folium.Map(location=[25.2744, 133.7751], tiles="Stamen Terrain", zoom_start=4)

# refer to the folium documentations on more information on how to plot aggregated data.
c = folium.Choropleth(
    geo_data=geoJSON, # geoJSON 
    name='choropleth', # name of plot
    data=df_population, # data source
    columns=['SA2_code', 'population'], # the columns required
    key_on='properties.SA2_CODE21', # this is from the geoJSON's properties
    fill_color='YlOrRd', # color scheme
    nan_fill_color='white',
    legend_name='Population'
)

c.add_to(_map)
_map

In [ ]:
raw_data = "../data/ABS/"

df_income = pd.read_excel(raw_data+"annual income by SA2 regions.xlsx")

In [ ]:
df_income.drop(np.arange(0,8), inplace=True)
df_income.drop(np.arange(2303,2306), inplace=True)

df_income.drop(df_income.columns[[2,3,4,5,7,8,9,10,12,13,14,15,17,18,19,20,22,23,24,25,27]], axis=1, inplace=True)
df_income = df_income.reset_index(drop=True)
df_income.columns = ['SA2_code', 'sa2_name', 'total_earners', 'median_age', 'income_sum', 'income_median', 'income_mean']
df_income

,SA2_code,sa2_name,total_earners,median_age,income_sum,income_median,income_mean
0,101021007,Braidwood,2361,51,120763285,41593,51149
1,101021008,Karabar,5100,42,338308979,61777,66335
2,101021009,Queanbeyan,6697,39,441160946,60119,65874
3,101021010,Queanbeyan - East,3393,40,237035656,63051,69860
4,101021011,Queanbeyan Region,12821,44,1050285692,71420,81919
...,...,...,...,...,...,...,...
2290,801101137,Molonglo,np,np,np,np,np
2291,801101138,Molonglo - North,np,np,np,np,np
2292,801101139,Wright,2121,35,182421537,79150,86007
2293,801111140,ACT - South West,355,40,26069449,64227,73435


In [ ]:
df_merge = df_income.merge(df_population, on='SA2_code', how='inner')
df_merge

,SA2_code,sa2_name,total_earners,median_age,income_sum,income_median,income_mean,population
0,101021007,Braidwood,2361,51,120763285,41593,51149,4330.0
1,101021008,Karabar,5100,42,338308979,61777,66335,8546.0
2,101021009,Queanbeyan,6697,39,441160946,60119,65874,11370.0
3,101021010,Queanbeyan - East,3393,40,237035656,63051,69860,5093.0
4,101021012,Queanbeyan West - Jerrabomberra,8476,44,725602722,73851,85607,12743.0
...,...,...,...,...,...,...,...,...
2142,801101136,Denman Prospect,416,38,41056005,90337,98692,2719.0
2143,801101137,Molonglo,np,np,np,np,np,0.0
2144,801101139,Wright,2121,35,182421537,79150,86007,3806.0
2145,801111140,ACT - South West,355,40,26069449,64227,73435,554.0


In [ ]:
df_merge = df_merge[df_merge["total_earners"] != "np"]
df_merge = df_merge[df_merge["population"] != 0.0]
df_merge

,SA2_code,sa2_name,total_earners,median_age,income_sum,income_median,income_mean,population
0,101021007,Braidwood,2361,51,120763285,41593,51149,4330.0
1,101021008,Karabar,5100,42,338308979,61777,66335,8546.0
2,101021009,Queanbeyan,6697,39,441160946,60119,65874,11370.0
3,101021010,Queanbeyan - East,3393,40,237035656,63051,69860,5093.0
4,101021012,Queanbeyan West - Jerrabomberra,8476,44,725602722,73851,85607,12743.0
...,...,...,...,...,...,...,...,...
2141,801101135,Coombs,2270,35,179834174,74179,79222,4834.0
2142,801101136,Denman Prospect,416,38,41056005,90337,98692,2719.0
2144,801101139,Wright,2121,35,182421537,79150,86007,3806.0
2145,801111140,ACT - South West,355,40,26069449,64227,73435,554.0


In [ ]:
# 选择计算地区总年收入/总人口数
df_merge['income_per_person'] = df_merge['income_sum'].astype(float) / df_merge['population']

In [ ]:
df_merge.describe()

,population,income_per_person
count,2097.000000,2.097000e+03
mean,10382.612780,4.119961e+04
std,6199.140647,8.826728e+04
min,3.000000,-1.285210e+03
25%,5358.000000,2.743243e+04
50%,9262.000000,3.378589e+04
75%,14829.000000,4.230943e+04
max,26871.000000,3.541638e+06


In [ ]:
df_merge = df_merge[df_merge["income_per_person"] > 0]
df_merge = df_merge[df_merge["income_per_person"] < 80000]
df_merge = df_merge[['SA2_code', 'income_per_person']]
df_merge.describe()

,income_per_person
count,2037.000000
mean,35361.189159
std,12046.052007
min,2135.836833
25%,27318.610893
50%,33498.627704
75%,41352.242055
max,79063.731287


In [ ]:
_map = folium.Map(location=[25.2744, 133.7751], tiles="Stamen Terrain", zoom_start=4)

# refer to the folium documentations on more information on how to plot aggregated data.
c = folium.Choropleth(
    geo_data=geoJSON, # geoJSON 
    name='choropleth', # name of plot
    data=df_merge, # data source
    columns=['SA2_code', 'income_per_person'], # the columns required
    key_on='properties.SA2_CODE21', # this is from the geoJSON's properties
    fill_color='YlOrRd', # color scheme
    nan_fill_color='white',
    legend_name='Average Income'
)

c.add_to(_map)
_map